# Compare metrics between the models

In [1]:
from IPython.display import clear_output

#!pip install datasets transformers rouge_score nltk
!pip install datasets transformers rouge-score nltk
# rouge-score is the google version
!pip install pyarrow
!pip install -q sentencepiece

clear_output()

In [2]:
from IPython.display import clear_output
import os
import re
import time
from tqdm.notebook import trange, tqdm
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt

# nlp stuff
import nltk
nltk.download('punkt')

# tf stuff
import tensorflow_datasets as tfds 
import tensorflow as tf
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration # pegasus
from transformers import BartTokenizer, TFBartForConditionalGeneration # bart

# pytorch dataset types
import datasets
from datasets.dataset_dict import DatasetDict
from datasets import Dataset, load_metric, load_dataset

# pytorch bart stuff
import torch
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer

# clear_output()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:

metric = load_metric("rouge")

ImportError: ignored

In [ ]:
# try a diff package for rouge
# !pip install rouge-score # google package version
# from rouge_score import rouge_scorer
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3', 'rougeL'], use_stemmer=True)
#scores = scorer.score(target=y, prediction=yhat)
#scores

In [28]:
# specify your path to the repo here:
repo_path = '/content/gdrive/MyDrive/w266/w266_reddit_summarization'

from google.colab import drive
drive.mount('/content/gdrive')

# baseline bart
baseline_preds = pd.read_parquet(os.path.join(repo_path, 'data/model_outputs/bart_preds/round2/bart_baseline_preds.parquet'))
baseline_preds.columns = ['content', 'y', 'yhat_baseline']
baseline_preds = baseline_preds.sort_values(['content', 'y'])
baseline_preds.reset_index(drop=True, inplace=True)

# finetuned
df1 = pd.read_parquet(os.path.join(repo_path, 'data/model_outputs/bart_preds/round2/bart_preds_advice.parquet'))
df1['subreddit_group'] = 'advice_story'
df2 = pd.read_parquet(os.path.join(repo_path, 'data/model_outputs/bart_preds/round2/bart_preds_media.parquet'))
df2['subreddit_group'] = 'media_lifestyle_sports'
df3 = pd.read_parquet(os.path.join(repo_path, 'data/model_outputs/bart_preds/round2/bart_preds_gaming.parquet'))
df3['subreddit_group'] = 'gaming'
df4 = pd.read_parquet(os.path.join(repo_path, 'data/model_outputs/bart_preds/round2/bart_preds_other.parquet'))
df4['subreddit_group'] = 'other'

# finetuned bart
finetuned_preds = pd.concat([df1, df2, df3, df4], ignore_index=True)
finetuned_preds.columns = ['content', 'y', 'yhat_finetune', 'subreddit_group']
finetuned_preds = finetuned_preds.sort_values(['content', 'y'])
finetuned_preds.reset_index(drop=True, inplace=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [38]:
df = pd.merge(baseline_preds, finetuned_preds[['yhat_finetune', 'subreddit_group']], left_index=True, right_index=True, how='inner')

# Metrics

In [40]:
# Advice/Story

baseline_metrics = metric.compute(predictions=df[df['subreddit_group' == 'advice_story']]['yhat_baseline'].tolist(), references=df[df['subreddit_group' == 'advice_story']]['y'].tolist())
finetune_metrics = metric.compute(predictions=df[df['subreddit_group' == 'advice_story']]['yhat_finetune'].tolist(), references=df[df['subreddit_group' == 'advice_story']]['y'].tolist())

print("Baseline:")
print(baseline_metrics)

print("\n\nBaseline:")
print(finetune_metrics)

ImportError: ignored

In [ ]:
# compute metrics for each genre